In [89]:
import os.path
import sys

import altair as alt
import bqplot
import ipywidgets
from IPython.display import display, HTML
import pandas as pd
import numpy as np

from solution import concentratedsolar
from solution import landfillmethane
from solution import solarpvroof
from solution import solarpvutil
import ui.dd_nb as dd

pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

options = ['solarpvutil:' + str(scenario) for scenario in solarpvutil.scenarios.keys()]
options += ['solarpvroof:' + str(scenario) for scenario in solarpvroof.scenarios.keys()]
options += ['landfillmethane:' + str(scenario) for scenario in landfillmethane.scenarios.keys()]
options += ['concentratedsolar:' + str(scenario) for scenario in concentratedsolar.scenarios.keys()]
solution_choices = ipywidgets.SelectMultiple(
    options=sorted(options, key=dd.scenario_sort_key), description='Scenarios',
    disabled=False, rows=8, layout=ipywidgets.Layout(width='500px')
)
solution_choices

SelectMultiple(description='Scenarios', layout=Layout(width='500px'), options=('concentratedsolar:PDS-4p2050-D…

In [113]:
# Set CSS properties for th elements in dataframe
th_props = [
  ('font-size', '13px'),
  ('text-align', 'center'),
  ('font-weight', 'bold'),
  ('color', '#6d6d6d'),
  ('background-color', '#f7f7f9')
  ]

# Set CSS properties for td elements in dataframe
td_props = [
  ('font-size', '12px'),
  ('font-family', 'monospace')
  ]

# Set table styles
dataframe_styles = [
  dict(selector="th", props=th_props),
  dict(selector="td", props=td_props)
  ]

In [169]:
#%%prun -s cumulative -l 200
if len(solution_choices.value):
    solutions = []
    for val in solution_choices.value:
        soln, scenario = val.split(':')
        if soln == 'solarpvutil':
            solutions.append(solarpvutil.SolarPVUtil(scenario=scenario))
        elif soln == 'solarpvroof':
            solutions.append(solarpvroof.SolarPVRoof(scenario=scenario))
        elif soln == 'landfillmethane':
            solutions.append(landfillmethane.LandfillMethane(scenario=scenario))
        elif soln == 'concentratedsolar':
            solutions.append(concentratedsolar.ConcentratedSolar(scenario=scenario))
else:
    solutions = [solarpvutil.SolarPVUtil()]

# ------------------ Detailed Results -----------------

def blue_label(text):
    o = ipywidgets.Output()
    div = '<div style="background-color:LightSkyBlue;border:1px solid black;font-weight:bold;text-align:center;font-size:0.875em;">'
    with o:
        display(HTML(div + text + '</div>'))
    return o

def fullname(s):
    return type(s).__name__ + ':' + s.scenario


key_results_label = blue_label('The Key Adoption Results')

# --- Unit Adoption ---

unit_adoption_text = []
for s in solutions:
    pds_vs_ref = s.ua.soln_pds_tot_iunits_reqd().loc[2050, 'World'] - s.ua.soln_ref_tot_iunits_reqd().loc[2050, 'World']
    unit_adoption_text.append([fullname(s), pds_vs_ref, s.ua.soln_net_annual_funits_adopted().loc[2050, 'World'] ])
unit_adoption_text.append(['', 'TW', 'TWh'])
unit_adoption_columns = ['Scenario', 'Implementation Unit Adoption Increase in 2050 (PDS vs REF)',
                         'Functional Unit Adoption Increase in 2050 (PDS vs REF)']
unit_adoption_heading = ipywidgets.Output()
with unit_adoption_heading:
    df = pd.DataFrame(unit_adoption_text, columns=unit_adoption_columns)
    display(HTML(df.style.set_table_styles(dataframe_styles).hide_index().render()))
        
# --- Adoption ---

adoption_text = []
for s in solutions:
    adoption_text.append([fullname(s),
                          s.ua.soln_pds_tot_iunits_reqd().loc[2050, 'World'],
                          s.ht.soln_pds_funits_adopted().loc[2050, 'World'],
                          '', '', ''])
adoption_text.append(['', 'TW', 'TWh', '%', '%', '%'])
adoption_columns = ['Scenario', 'Global Solution Implementation Units in 2050',
                    'Global Solution Functional Units in 2050', 'Global Solution Adoption Share - 2014',
                    'Global Solution Adoption Share - 2020', 'Global Solution Adoption Share - 2050']
adoption_heading = ipywidgets.Output()
with adoption_heading:
    df = pd.DataFrame(adoption_text, columns=adoption_columns)
    display(HTML(df.style.set_table_styles(dataframe_styles).hide_index().render()))


adoption_iunit_chart = ipywidgets.Output()
with adoption_iunit_chart:
    df = pd.DataFrame()
    for s in solutions:
        df[fullname(s)] = s.ua.soln_pds_tot_iunits_reqd().loc[2020:2050, 'World']
    iunit_df = df.reset_index().melt('Year', value_name='iunits', var_name='solution')
    iunit_chart = alt.Chart(iunit_df, width=400).mark_line().encode(
        y='iunits',
        x='Year:O',
        color=alt.Color('solution', legend=alt.Legend(orient='top-left')),
        tooltip=['solution', 'iunits', 'Year'],
    ).properties(
        title='World Adoption - Implementation Units (TW)'
    ).interactive()
    display(iunit_chart)

adoption_funit_chart = ipywidgets.Output()
with adoption_funit_chart:
    df = pd.DataFrame()
    for s in solutions:
        df['Total Market-REF:'+fullname(s)] = s.tm.ref_tam_per_region().loc[2020:2050, 'World']
        df['Total Market-PDS:'+fullname(s)] = s.tm.pds_tam_per_region().loc[2020:2050, 'World']
        df['Solution-REF:'+fullname(s)] = s.ht.soln_ref_funits_adopted().loc[2020:2050, 'World']
        df['Solution-PDS:'+fullname(s)] = s.ht.soln_pds_funits_adopted().loc[2020:2050, 'World']
    funit_df = df.reset_index().melt('Year', value_name='funits', var_name='solution')
    funit_chart = alt.Chart(funit_df, width=400).mark_line().encode(
        y='funits',
        x=alt.X('Year', type='ordinal', scale=alt.Scale(domain=list(range(2015, 2056)))),
        color=alt.Color('solution', legend=alt.Legend(orient='top-right')),
        tooltip=['solution', 'funits', 'Year'],
    ).properties(
        title='World Growth of Solution - Functional Units (TWh)'
    ).interactive()
    display(funit_chart)

adoption_graphs = ipywidgets.HBox(children=[adoption_iunit_chart, adoption_funit_chart])

# --- Financial ---

financial_results_label = blue_label('The Key Financial Results</div>')

financial_text = []
for s in solutions:
    marginal_first_cost = (s.fc.soln_pds_annual_world_first_cost().loc[:2050].sum() - \
        s.fc.soln_ref_annual_world_first_cost().loc[:2050].sum() - \
        s.fc.conv_ref_annual_world_first_cost().loc[:2050].sum()) / 10**9
    net_operating_savings = ((s.oc.conv_ref_cumulative_operating_cost().loc[2050] - \
                             s.oc.conv_ref_cumulative_operating_cost().loc[2020]) - \
                            (s.oc.soln_pds_cumulative_operating_cost().loc[2050] - \
                             s.oc.soln_pds_cumulative_operating_cost().loc[2020])) / 10**9
    lifetime_operating_savings = s.oc.lifetime_cost_forecast()['Marginal Operating Cost Savings'].sum() / 10**9
    financial_text.append([fullname(s), marginal_first_cost, net_operating_savings, lifetime_operating_savings])
financial_text.append(['', 'Billion USD', 'Billion USD', 'Billion USD'])
financial_columns = ['Scenario', 'Marginal First Cost 2020-2050',
                     'Net Operating Savings 2020-2050', 'Lifetime Operating Savings 2020-2050']

financial_heading = ipywidgets.Output()
with financial_heading:
    df = pd.DataFrame(financial_text, columns=financial_columns)
    display(HTML(df.style.set_table_styles(dataframe_styles).hide_index().render()))

cost_graphs = ipywidgets.Output()
with cost_graphs:
    df = pd.DataFrame()
    for s in solutions:
        df['Solution Operating Costs/Savings:'+fullname(s)] = s.oc.soln_pds_annual_operating_cost().loc[2020:2050] / 1000000000
        df['Conventional Operating Costs/Savings:'+fullname(s)] = s.oc.conv_ref_annual_operating_cost().loc[2020:2050] / 1000000000
    cost_df = df.reset_index().melt('Year', value_name='costs', var_name='solution')
    cost_chart = alt.Chart(cost_df, width=350).mark_line().encode(
        y=alt.Y('costs', title='US$B'),
        x=alt.X('Year', type='ordinal', scale=alt.Scale(domain=list(range(2010, 2061)))),
        color=alt.Color('solution', legend=alt.Legend(orient='top-left')),
        tooltip=['solution', 'costs', 'Year'],
    ).properties(
        title='World Operating Cost Difference'
    ).interactive()
    display(cost_chart)


# --- Climate ---

climate_results_label = blue_label('The Key Climate Results</div>')

climate_text = []
for s in solutions:
    climate_text.append([fullname(s), (s.c2.co2eq_mmt_reduced().loc[:, 'World'].sum() / 1000)])
climate_text.append(['', 'Gt CO2 (2020-2050)'])
climate_columns = ['Scenario', 'Total Emissions Reduction']

climate_heading = ipywidgets.Output()
with climate_heading:
    df = pd.DataFrame(climate_text, columns=climate_columns)
    display(HTML(df.style.set_table_styles(dataframe_styles).hide_index().render()))

reduction_graph = ipywidgets.Output()
with reduction_graph:
    df = pd.DataFrame()
    for s in solutions:
        df[fullname(s)] = s.c2.co2eq_mmt_reduced().loc[2020:2050, 'World'].cumsum().mul(0.001)
    reduction_df = df.reset_index().melt('Year', value_name='reduction', var_name='solution')
    reduction_chart = alt.Chart(reduction_df, width=300).mark_line().encode(
        y='reduction',
        x=alt.X('Year', type='ordinal'),
        color=alt.Color('solution', legend=alt.Legend(orient='top-left')),
        tooltip=['solution', 'reduction', 'Year'],
    ).properties(
        title='World Cumulative Emissions Reductions (Gigatons CO2-eq)'
    ).interactive()
    display(reduction_chart)

concentration_graph = ipywidgets.Output()
with concentration_graph:
    df = pd.DataFrame()
    for s in solutions:
        df[fullname(s)] = s.c2.co2eq_ppm_calculator().loc[2020:2050, 'CO2-eq PPM']
    concentration_df = df.reset_index().melt('Year', value_name='concentration', var_name='solution')
    concentration_chart = alt.Chart(concentration_df, width=300).mark_line().encode(
        y='concentration',
        x=alt.X('Year', type='ordinal'),
        color=alt.Color('solution', legend=alt.Legend(orient='top-left')),
        tooltip=['solution', 'concentration', 'Year'],
    ).properties(
        title='World Cumulative Emissions Reductions (Gigatons CO2-eq)'
    ).interactive()
    display(concentration_chart)

emissions_graphs = ipywidgets.HBox(children=[reduction_graph, concentration_graph])

detailed_results = ipywidgets.Output()
with detailed_results:
    display(ipywidgets.VBox([key_results_label, unit_adoption_heading, adoption_heading, adoption_graphs,
                             financial_results_label, financial_heading, cost_graphs,
                             climate_results_label, climate_heading, emissions_graphs]))

# --------------- Variable Meta Analysis --------------

variable_meta_analysis = ipywidgets.Output()
with variable_meta_analysis:
    children = []
    for s in solutions:
        for (name, v) in s.VMAs:
            vma_widget = ipywidgets.Output()
            with vma_widget:
                display(HTML(v.source_data.drop(['Link', 'Source Validation Code', 'License Code'], axis=1).fillna('').to_html()))
            summary_table = ipywidgets.Output()
            with summary_table:
                (mean, high, low) = v.avg_high_low()
                df = pd.DataFrame([[mean, high, low]], columns=['Mean', 'High', 'Low'])
                display(HTML(df.style.format("{:.01f}").set_properties(**{'width': '10em'}).set_table_styles(dataframe_styles).hide_index().render()))

            accordion = ipywidgets.Accordion(children=[vma_widget], layout=ipywidgets.Layout(width='90%'))
            accordion.set_title(0, name)
            vma_widget = ipywidgets.Output()
            with vma_widget:
                display(ipywidgets.HBox(children=[summary_table, accordion]))
            children.append(vma_widget)
    display(ipywidgets.VBox(children=children))

# ------------------ First Cost -----------------

children = []
for s in solutions:
    df = pd.concat([s.fc.soln_pds_install_cost_per_iunit().loc[2020:2050],
                    s.fc.soln_ref_install_cost_per_iunit().loc[2020:2050],
                    s.fc.conv_ref_install_cost_per_iunit().loc[2020:2050]],
                   axis=1)
    df.columns = ['SOLN-PDS', 'SOLN-REF', 'CONV-REF']
    df[['SOLN-PDS', 'SOLN-REF', 'CONV-REF']] /= 1000000
    fc_table = ipywidgets.Output()
    with fc_table:
        display(HTML(df.style.format({'SOLN-PDS':'{:.02f}', 'SOLN-REF':'{:.02f}', 'CONV-REF':'{:.02f}'}).set_table_styles(dataframe_styles).render()))
    fc_chart = ipywidgets.Output()
    with fc_chart:
        melted_df = df.reset_index().melt('Year', value_name='cost', var_name='column')
        melted_chart = alt.Chart(melted_df, width=300).mark_line().encode(
            y='cost',
            x='Year:O',
            color=alt.Color('column'),
            tooltip=['column', 'cost', 'Year'],
        ).properties(
            title='First Cost (US$M)'
        ).interactive()
        display(melted_chart)
    children.append(ipywidgets.HBox([fc_table, fc_chart]))

first_cost = ipywidgets.Accordion(children=children)
for i, s in enumerate(solutions):
    first_cost.set_title(i, fullname(s))

# ------------------ Operating Cost -----------------

children = []
for s in solutions:
    df = pd.concat([s.oc.soln_pds_annual_operating_cost().loc[2020:2050],
                    s.oc.conv_ref_annual_operating_cost().loc[2020:2050],
                    s.oc.marginal_annual_operating_cost().loc[2020:2050]],
                   axis=1)
    df.columns = ['PDS', 'REF', 'Marginal']
    df[['PDS', 'REF', 'Marginal']] /= 1000000
    oc_table = ipywidgets.Output()
    with oc_table:
        display(HTML(df.style.format({'PDS':'{:.02f}', 'REF':'{:.02f}', 'Marginal':'{:.02f}'}).set_table_styles(dataframe_styles).render()))
    oc_chart = ipywidgets.Output()
    with oc_chart:
        melted_df = df.reset_index().melt('Year', value_name='cost', var_name='column')
        melted_chart = alt.Chart(melted_df, width=300).mark_line().encode(
            y='cost',
            x='Year:O',
            color=alt.Color('column'),
            tooltip=['column', 'cost', 'Year'],
        ).properties(
            title='Operating Costs (US$M)'
        ).interactive()
        display(melted_chart)
    children.append(ipywidgets.HBox([oc_table, oc_chart]))

operating_cost = ipywidgets.Accordion(children=children)
for i, s in enumerate(solutions):
    operating_cost.set_title(i, fullname(s))

# ------------------ Adoption Data -----------------

children = []
for s in solutions:
    df = pd.concat([s.ht.soln_pds_funits_adopted().loc[2020:2050, 'World'],
                    s.ht.soln_ref_funits_adopted().loc[2020:2050, 'World']],
                   axis=1)
    df.columns = ['PDS', 'REF']
    ad_table = ipywidgets.Output()
    with ad_table:
        display(HTML(df.style.format({'PDS':'{:.02f}', 'REF':'{:.02f}'}).set_table_styles(dataframe_styles).render()))
    ad_chart = ipywidgets.Output()
    with ad_chart:
        melted_df = df.reset_index().melt('Year', value_name='adoption', var_name='column')
        melted_chart = alt.Chart(melted_df, width=300).mark_line().encode(
            y='adoption',
            x='Year:O',
            color=alt.Color('column'),
            tooltip=['column', 'adoption', 'Year'],
        ).properties(
            title='Adoption (IUnits)'
        ).interactive()
        display(melted_chart)
    children.append(ipywidgets.HBox([ad_table, ad_chart]))

adoption_data = ipywidgets.Accordion(children=children)
for i, s in enumerate(solutions):
    adoption_data.set_title(i, fullname(s))

# ------------------ CO2 Reduction -----------------

children = []
for s in solutions:
    df = pd.concat([s.c2.co2_mmt_reduced().loc[2020:2050, 'World'],
                    s.c2.co2eq_mmt_reduced().loc[2020:2050, 'World']],
                   axis=1)
    df.columns = ['CO2', 'CO2eq']
    c2_table = ipywidgets.Output()
    with c2_table:
        display(HTML(df.style.format({'CO2':'{:.02f}', 'CO2eq':'{:.02f}'}).set_table_styles(dataframe_styles).render()))
    c2_chart = ipywidgets.Output()
    with c2_chart:
        melted_df = df.reset_index().melt('Year', value_name='mmt', var_name='column')
        melted_chart = alt.Chart(melted_df, width=300).mark_line().encode(
            y='mmt',
            x='Year:O',
            color=alt.Color('column'),
            tooltip=['column', 'mmt', 'Year'],
        ).properties(
            title='CO2 Reduction (MMT)'
        ).interactive()
        display(melted_chart)
    children.append(ipywidgets.HBox([c2_table, c2_chart]))

co2_calcs = ipywidgets.Accordion(children=children)
for i, s in enumerate(solutions):
    co2_calcs.set_title(i, fullname(s))

In [170]:
solutions = pd.read_csv(os.path.join('data', 'overview', 'solutions.csv'),
                                    index_col=False, skipinitialspace=True, header=0,
                                    skip_blank_lines=True, comment='#')
soln_results = pd.read_csv(os.path.join('data', 'overview', 'soln_results.csv'),
                                    index_col=False, skipinitialspace=True, header=0,
                                    skip_blank_lines=True, comment='#')
solutions = solutions.merge(soln_results, on='Solution', how='left')

colormap = {
    'Materials': 'RebeccaPurple',
    'Electricity Generation': 'Peru',
    'Food': 'FireBrick',
    'Land Use': 'Green',
    'Women and Girls': 'DarkGoldenRod',
    'Transport': 'Teal',
    'Buildings and Cities': 'SteelBlue',
}

def get_color(sector):
    return colormap.get(sector, 'Beige')

soln_data = solutions['CO2eq'].tolist()
soln_labels = solutions['Solution'].tolist()
soln_colors = solutions['Sector'].apply(get_color).tolist()
soln_tt = bqplot.Tooltip(fields=['label', 'size'], formats=['', '0.1f'], labels=['Name', 'GTons-CO2'])
soln_pie = bqplot.Pie(sizes=soln_data, display_labels='none', labels=soln_labels, sort=True, tooltip=soln_tt,
                 display_values=True, values_format='.1f', radius=150, inner_radius=130,
                 colors=soln_colors, stroke='Gainsboro',
                 tooltip_style = {'opacity': 0.7, 'background-color': 'White', 'border': 'none',
                               'border-collapse': 'collapse'})

sectors = solutions.pivot_table(index='Sector',aggfunc=sum)
solutions['SectorCO2eq'] = solutions.apply(lambda row: sectors.loc[row['Sector'], 'CO2eq'], axis=1)
sctr_data = sectors['CO2eq'].tolist()
sctr_labels = sectors.index.tolist()
sctr_colors = [get_color(v) for v in sctr_labels]
sctr_tt = bqplot.Tooltip(fields=['label', 'size'], formats=['', '0.1f'], labels=['Name', 'GTons-CO2eq'])
sctr_pie = bqplot.Pie(sizes=sctr_data, display_labels='none', labels=sctr_labels, sort=True, tooltip=sctr_tt,
                 display_values=True, values_format='.1f', radius=118, inner_radius=98,
                 colors=sctr_colors, stroke='Gainsboro')
sctr_pie.tooltip_style = {'opacity': 0.7, 'background-color': 'White', 'border': 'none',
                               'border-collapse': 'collapse'}
solution_chart = bqplot.Figure(marks=[soln_pie, sctr_pie], animation_duration=1000)
solution_chart.layout.width = '430px'
solution_chart.background_style = {'fill': 'White'}

solution_list = ipywidgets.Output()
with solution_list:
    display(HTML((solutions.sort_values(['SectorCO2eq', 'CO2eq'], ascending=False)
                  .loc[:,['Solution', 'Sector', 'CO2eq', 'DirName']].fillna('')
                  .style.format({'CO2eq':'{:.02f}'}).set_table_styles(dataframe_styles)
                  .hide_index().render())))

# Treemap of solutions, constructed using https://vega.github.io/vega/docs/
elements = {'root': {'id': 1}}
id = 2
for row in sectors.itertuples(index=True):
    name = getattr(row, 'Index')
    elements[name] = {'id': id, 'name': name, 'parent': 1, 'color': get_color(name)}
    id += 1
for row in solutions.itertuples(index=False):
    name = getattr(row, "Solution")
    sector = getattr(row, "Sector")
    co2eq = getattr(row, "CO2eq")
    parent_id = elements.get(sector, 1)['id']
    elements[name] = {'id': id, 'name': name, 'parent': parent_id, 'size': co2eq}
    id += 1

solution_treemap = ipywidgets.Output()
with solution_treemap:
    display({'application/vnd.vega.v4+json': {
      "$schema": "https://vega.github.io/schema/vega/v4.json",
      "width": 400,
      "height": 800,
      "padding": 2.5,
      "autosize": "none",

      "signals": [
        {
          "name": "layout", "value": "squarify",
        },
        {
          "name": "aspectRatio", "value": 1.6,
        }
      ],

      "data": [
        {
          "name": "drawdown",
          "values": list(elements.values()),
          "transform": [
            {
              "type": "stratify",
              "key": "id",
              "parentKey": "parent"
            },
            {
              "type": "treemap",
              "field": "size",
              "sort": {"field": "value"},
              "round": True,
              "method": {"signal": "layout"},
              "ratio": {"signal": "aspectRatio"},
              "size": [{"signal": "width"}, {"signal": "height"}]
            }
          ]
        },
        {
          "name": "nodes",
          "source": "drawdown",
          "transform": [{ "type": "filter", "expr": "datum.children" }]
        },
        {
          "name": "leaves",
          "source": "drawdown",
          "transform": [{ "type": "filter", "expr": "!datum.children" }]
        }
      ],

      "scales": [
        {
          "name": "color",
          "type": "ordinal",
          "domain": list(colormap.keys()),
          "range": list(colormap.values())
        },
      ],

      "marks": [
        {
          "type": "rect",
          "from": {"data": "nodes"},
          "interactive": False,
          "encode": {
            "enter": {
              "fill": {"scale": "color", "field": "name"}
            },
            "update": {
              "x": {"field": "x0"},
              "y": {"field": "y0"},
              "x2": {"field": "x1"},
              "y2": {"field": "y1"}
            }
          }
        },
        {
          "type": "rect",
          "from": {"data": "leaves"},
          "encode": {
            "enter": {
              "stroke": {"value": "#fff"},
              "tooltip": {"signal": "{title: datum.name, 'CO2eq': datum.size + ' Gigatons'}"}
            },
            "update": {
              "x": {"field": "x0"},
              "y": {"field": "y0"},
              "x2": {"field": "x1"},
              "y2": {"field": "y1"},
              "fill": {"value": "transparent"}
            },
            "hover": {
              "fill": {"value": "gray"}
            }
          }
        },
        {
          "type": "text",
          "from": {"data": "nodes"},
          "interactive": False,
          "encode": {
            "enter": {
              "font": {"value": "Helvetica Neue, Arial"},
              "align": {"value": "center"},
              "baseline": {"value": "middle"},
              "fill": {"value": "#fff"},
              "text": {"field": "name"},
              "fontSize": {"value": 18},
              "fillOpacity": {"value": 1.0},
              "angle": {"value": -62.0}
            },
            "update": {
              "x": {"signal": "0.5 * (datum.x0 + datum.x1)"},
              "y": {"signal": "0.5 * (datum.y0 + datum.y1)"}
            }
          }
        }
      ]
    }
    }, raw=True)

charts = ipywidgets.VBox(children=[solution_chart, solution_treemap])
overview = ipywidgets.HBox(children=[solution_list, charts])

In [171]:
# ------------------ Create tabs -----------------

tabs = ipywidgets.Tab(children=[overview, detailed_results, variable_meta_analysis, adoption_data, first_cost, operating_cost, co2_calcs])
tabs.set_title(0, "Overview")
tabs.set_title(1, "Detailed Results")
tabs.set_title(2, "Variable Meta-Analysis")
tabs.set_title(3, "Adoption Data")
tabs.set_title(4, "First Cost")
tabs.set_title(5, "Operating Cost")
tabs.set_title(6, "CO2 Reduction")
display(tabs)